In [2]:
import os
os.makedirs("models", exist_ok=True)
os.makedirs("training_logs", exist_ok=True)

In [3]:
%%writefile requirements.txt
numpy
pandas
matplotlib
tensorflow
scikit-learn
seaborn

Writing requirements.txt


In [10]:
%%writefile neural_network_scratch.py
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size):
        # He Initialization (Crucial for high accuracy)
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2/input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2/hidden_size)
        self.b2 = np.zeros((1, output_size))

    def relu(self, Z):
        return np.maximum(0, Z)

    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return expZ / np.sum(expZ, axis=1, keepdims=True)

    def forward(self, X):
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.softmax(self.Z2)
        return self.A2

    def backward(self, X, y_true, learning_rate=0.1):
        m = y_true.shape[0]
        dZ2 = self.A2 - y_true
        dW2 = (1/m) * np.dot(self.A1.T, dZ2)
        db2 = (1/m) * np.sum(dZ2, axis=0, keepdims=True)
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * (self.Z1 > 0)
        dW1 = (1/m) * np.dot(X.T, dZ1)
        db1 = (1/m) * np.sum(dZ1, axis=0, keepdims=True)
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

    def train(self, X, y, epochs=1000, lr=0.1):
        print(f"--- Training NN from Scratch on Digits Data ---")
        final_acc = 0
        for i in range(epochs):
            y_pred = self.forward(X)
            self.backward(X, y, learning_rate=lr)

            if i % 100 == 0:
                predictions = np.argmax(y_pred, axis=1)
                true_labels = np.argmax(y, axis=1)
                final_acc = np.mean(predictions == true_labels)
                print(f"Epoch {i}: Accuracy = {final_acc*100:.2f}%")
        return final_acc

if __name__ == "__main__":
    # 1. Load Real Data
    digits = load_digits()
    X = digits.data
    y = digits.target.reshape(-1, 1)

    # 2. Scale Data (StandardScaler is vital for convergence)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    encoder = OneHotEncoder(sparse_output=False)
    y_onehot = encoder.fit_transform(y)

    # 3. Train
    nn = SimpleNN(input_size=64, hidden_size=64, output_size=10)
    final_acc = nn.train(X, y_onehot, epochs=2000, lr=0.5)

    # 4. Save Proof
    with open("results_scratch.txt", "w") as f:
        f.write(f"Manual NN Accuracy: {final_acc*100:.2f}%")
    print(f"\n[Success] Results saved to results_scratch.txt")

Overwriting neural_network_scratch.py


In [11]:
!python neural_network_scratch.py

--- Training NN from Scratch on Digits Data ---
Epoch 0: Accuracy = 14.58%
Epoch 100: Accuracy = 99.39%
Epoch 200: Accuracy = 99.83%
Epoch 300: Accuracy = 100.00%
Epoch 400: Accuracy = 100.00%
Epoch 500: Accuracy = 100.00%
Epoch 600: Accuracy = 100.00%
Epoch 700: Accuracy = 100.00%
Epoch 800: Accuracy = 100.00%
Epoch 900: Accuracy = 100.00%
Epoch 1000: Accuracy = 100.00%
Epoch 1100: Accuracy = 100.00%
Epoch 1200: Accuracy = 100.00%
Epoch 1300: Accuracy = 100.00%
Epoch 1400: Accuracy = 100.00%
Epoch 1500: Accuracy = 100.00%
Epoch 1600: Accuracy = 100.00%
Epoch 1700: Accuracy = 100.00%
Epoch 1800: Accuracy = 100.00%
Epoch 1900: Accuracy = 100.00%

[Success] Results saved to results_scratch.txt


In [12]:
%%writefile cnn_classifier.py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os

def train_cnn():
    print("--- Loading MNIST Data for CNN ---")
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Reshape and Scale
    X_train = X_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
    X_test = X_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

    print("--- Building CNN Architecture ---")
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print("--- Training CNN ---")
    history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

    # Evaluate
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Final Test Accuracy: {test_acc*100:.2f}%")

    # Save Model & Proof
    model.save("models/cnn_mnist.keras")

    with open("results_cnn.txt", "w") as f:
        f.write(f"CNN MNIST Accuracy: {test_acc*100:.2f}%")
    print("\n[Success] Results saved to results_cnn.txt")

if __name__ == "__main__":
    train_cnn()

Writing cnn_classifier.py


In [13]:
!python cnn_classifier.py

2026-02-04 16:53:31.201925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770224011.219964    5103 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770224011.225394    5103 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770224011.239237    5103 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770224011.239265    5103 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770224011.239268    5103 computation_placer.cc:177] computation placer alr

In [25]:
%%writefile transfer_learning.py
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, Input
import numpy as np
import os

# Ensure models directory exists
os.makedirs("models", exist_ok=True)

def run_transfer_learning():
    print("--- 1. Loading FULL CIFAR-10 Data ---")
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # Cast to float
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")

    print(f"Training on {len(X_train)} images...")

    print("--- 2. Building Architecture with Augmentation ---")

    inputs = Input(shape=(32, 32, 3))

    # Data Augmentation
    x = layers.RandomFlip("horizontal")(inputs)
    x = layers.RandomRotation(0.1)(x)

    # Resize to 96x96
    x = layers.Lambda(lambda img: tf.image.resize(img, (96, 96)))(x)

    # Preprocess
    x = layers.Lambda(lambda img: tf.keras.applications.mobilenet_v2.preprocess_input(img))(x)

    # Base Model
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=x)
    base_model.trainable = False

    # Head
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print("--- 3. Phase 1: Training Head ---")
    history = model.fit(X_train, y_train, epochs=8, batch_size=64, validation_split=0.2)

    print("--- 4. Phase 2: Fine-Tuning ---")
    base_model.trainable = True

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history_fine = model.fit(X_train, y_train, epochs=8, batch_size=64, validation_split=0.2)

    # Evaluate
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Final Test Accuracy: {test_acc*100:.2f}%")

    # --- THIS WAS THE BROKEN LINE ---
    model.save("models/transfer_learning_mobilenet.keras")

    with open("results_transfer.txt", "w") as f:
        f.write(f"Transfer Learning Accuracy: {test_acc*100:.2f}%")
    print("\n[Success] Results saved to results_transfer.txt")

if __name__ == "__main__":
    run_transfer_learning()

Overwriting transfer_learning.py


In [26]:
!python transfer_learning.py

2026-02-04 17:17:59.207949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770225479.225933   12137 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770225479.231520   12137 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770225479.245484   12137 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770225479.245508   12137 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770225479.245512   12137 computation_placer.cc:177] computation placer alr

In [27]:
%%writefile training_pipeline.py
import os

def run_pipeline():
    print("=======================================")
    print("   DAY 4: DEEP LEARNING PIPELINE      ")
    print("=======================================")

    # Step 1
    print("\n[Step 1] Running Manual Neural Network...")
    os.system("python neural_network_scratch.py")

    # Step 2
    print("\n[Step 2] Training CNN on MNIST...")
    os.system("python cnn_classifier.py")

    # Step 3
    print("\n[Step 3] Applying Transfer Learning...")
    os.system("python transfer_learning.py")

    print("\n=======================================")
    print("   PIPELINE COMPLETE. CHECK FILES.     ")
    print("=======================================")

if __name__ == "__main__":
    run_pipeline()

Writing training_pipeline.py
